In [20]:
import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt

df = pd.read_excel('questions.xlsx',header=None)

questions = pd.DataFrame(columns=['Question', 'Colour', 'Concept', 'Discipline', 'Place'])
scores = pd.DataFrame(columns=['Question', 'Colour', 'Concept', 'Discipline', 'Place'])

preferences = pd.read_csv('perference_transformed.csv')
user_preferences = preferences.iloc[1-1].to_dict()

decision_datas = pd.read_csv('decision.csv',header=None)
decision_data = decision_datas[decision_datas.iloc[:, 0] == 1]

groups = {
    'Colour': ['PURPLE', 'BLUE', 'GREEN', 'RED', 'ORANGE'],
    'Concept': ['RISK', 'HOPE', 'SAFETY', 'VITALITY', 'POWER'],
    'Discipline': ['LITERATURE', 'PHYSICS', 'MUSIC', 'HISTORY', 'GEOGRAPHY'],
    'Place': ['SEA', 'DESERT', 'CITY', 'MOUNTAIN', 'VILLAGE']
}

value_to_group = {
    value: group
    for group, values in groups.items()
    for value in values
}

for index, row in df.iterrows():
    question_num = row.iloc[0]
    groups = re.findall(r'([A-Z]+) - (\d+), ([A-Z]+) - (\d+), ([A-Z]+) - (\d+), ([A-Z]+) - (\d+)', row.iloc[1])
    if groups:
        groups = groups[0]
        question_row = {
            'Question': question_num,
            'Colour': groups[0],
            'Concept': groups[2],
            'Discipline': groups[4],
            'Place': groups[6]
        }
        score_row = {
            'Question': question_num,
            'Colour': int(groups[1]),
            'Concept': int(groups[3]),
            'Discipline': int(groups[5]),
            'Place': int(groups[7])
        }
        
        questions = pd.concat([questions, pd.DataFrame([question_row])], ignore_index=True)
        scores = pd.concat([scores, pd.DataFrame([score_row])], ignore_index=True)


In [7]:
# Obtain the options and score of the current question
def get_current_options(row, scores, index):
    return {
        'Colour': (row['Colour'], scores.iloc[index]['Colour']),
        'Concept': (row['Concept'], scores.iloc[index]['Concept']),
        'Discipline': (row['Discipline'], scores.iloc[index]['Discipline']),
        'Place': (row['Place'], scores.iloc[index]['Place'])
    }

# record the result
def record_choice(question_num, best_items, current_options, results):
    choices_str = ", ".join(best_items)

    scores = []
    for item in best_items:
        for _, (opt, score) in current_options.items():
            if opt == item:
                scores.append(str(score))
    scores_str = ", ".join(scores)

    result_row = {
        'Question': question_num,
        'Choice': choices_str,
        'Score': scores_str
    }
    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)
    return results

# Calculate the similarity between the predicted options of the current strategy and the actual options chosen by the subjects
# It is the degree of strategy fit
# "decision_choices" represents the answers given by the subjects, while "result_choices" represents the answers predicted by the strategy
def calculate_overlap(decision_choices, result_choices):
    overlap_count = sum(1 for i in range(len(decision_choices)) if decision_choices[i] in result_choices[i])
    overlap_rate = overlap_count / len(decision_choices)
    return overlap_rate

In [8]:
# Q-RL (Q-Reinforcement Learning)
# It is not a strict form of Q-learning, but merely uses the framework.
# Similar to the forgetting mechanism, the older the score, the lower its weight.
def evaluate_q_rl(decision_data: pd.DataFrame) -> tuple:
    
    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    learning_rate = 0.1
    discount_factor = 0.9

    # Initialize the Q-values of all possible options to 0.
    all_options = set()
    for _, row in questions.iterrows():
        all_options.update([
            row['Colour'], row['Concept'], 
            row['Discipline'], row['Place']
        ])
    q_values = {option: 0 for option in all_options}

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        # Find all the options with the highest Q values
        options_with_q = [(option, q_values[option]) for option, _ in current_options.values()]
        max_q = max(q for _, q in options_with_q)
        best_items = [option for option, q in options_with_q if q == max_q]
        
        # Update the Q values of all options
        for option, current_score in current_options.values():
            old_q = q_values[option]
            max_future_q = max(q_values[opt] for opt, _ in current_options.values())
            new_q = old_q + learning_rate * (current_score + discount_factor * max_future_q - old_q)
            q_values[option] = new_q
        
        results = record_choice(row['Question'], best_items, current_options, results)

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_q_rl(decision_data)

In [9]:
# R-Item (Item Recency)
# Make the selection directly based on the score of the previous round's options

def evaluate_R_Item(decision_data: pd.DataFrame) -> tuple:

    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    history = {}

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        # If history is empty, list all the options
        if not any(option in history for option, _ in current_options.values()):
            best_items = [option for option, _ in current_options.values()]
        else:
            options_with_history = []
            for option, current_score in current_options.values():
                if option in history:
                    options_with_history.append((option, history[option]))
            # Select the option for obtaining the highest score from the previous round
            max_score = max(score for _, score in options_with_history)
            best_items = [option for option, score in options_with_history if score == max_score]

        # Update History
        for option, current_score in current_options.values():
            history[option] = current_score

        results = record_choice(row['Question'], best_items, current_options, results)

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_R_Item(decision_data)

In [10]:
# R-Cat (Category Recency)
# Make your choice directly based on the group scores from the previous question

def evaluate_Simple_R_Cat(decision_data: pd.DataFrame) -> tuple:
    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    history = {}  # {category: last_score}

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        # If the category history is empty, select all categories
        if not any(category in history for category in current_options.keys()):
            best_categories = list(current_options.keys())
        else:
            categories_with_history = []
            for category, (option, current_score) in current_options.items():
                if category in history:
                    categories_with_history.append((category, history[category]))

            max_score = max(score for _, score in categories_with_history)
            best_categories = [category for category, score in categories_with_history if score == max_score]

        # Obtain all possible items based on all possible categories
        best_items = [current_options[cat][0] for cat in best_categories]

        # Update History
        for category, (option, current_score) in current_options.items():
            history[category] = current_score

        results = record_choice(row['Question'], best_items, current_options, results)

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_Simple_R_Cat(decision_data)

In [11]:
# LSC (Least Selected Category)
# Choose the group with the least number of previous selections

def evaluate_LSC(decision_data: pd.DataFrame) -> tuple:
    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    selection_count = {
    'Colour': 0,
    'Concept': 0,
    'Discipline': 0,
    'Place': 0
    }
    
    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        min_count = min(selection_count.values())
        least_selected = [category for category in current_options.keys() 
                        if selection_count[category] == min_count]
        best_items = [current_options[option][0] for option in least_selected]

        # Update History
        user_decision_item = decision_data.iloc[index, 2]
        selection_count[value_to_group[user_decision_item]] += 1

        results = record_choice(row['Question'], best_items, current_options, results)

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_LSC(decision_data)

In [12]:
# WSLS (Win-Stay, Lose-Shift)
# If the current option category score is greater than or equal to three, 
# then continue to select. 
# On the contrary, you can choose any other category.

def evaluate_WSLS(decision_data: pd.DataFrame) -> tuple:
    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    valid_groups = ['Colour', 'Concept', 'Discipline', 'Place']
    best_category = valid_groups

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)
        user_decision_groups = value_to_group[decision_data.iloc[index, 2]]

        best_items = [current_options[option][0] for option in best_category]
        results = record_choice(row['Question'], best_items, current_options, results)

        if current_options[user_decision_groups][1] < 3:
            best_category = [g for g in valid_groups if g != user_decision_groups]
        else:
            best_category = [user_decision_groups]

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())

    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_WSLS(decision_data)


In [13]:
# EV (Expected Value)
# Choose the option with the highest average expected value

def evaluate_EV(decision_data: pd.DataFrame) -> tuple:
    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    expectations = {}  # {option: (total, count)}

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        options_with_expectation = []
        for option, current_score in current_options.values():
            if option in expectations:
                total, count = expectations[option]
                avg_score = total / count
            else:
                avg_score = 0
            options_with_expectation.append((option, avg_score))

        max_avg = max(avg for _, avg in options_with_expectation)
        best_items = [option for option, avg in options_with_expectation if avg == max_avg]

        for option, current_score in current_options.values():
            if option in expectations:
                total, count = expectations[option]
                expectations[option] = (total + current_score, count + 1)
            else:
                expectations[option] = (current_score, 1)

        results = record_choice(row['Question'], best_items, current_options, results)

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_EV(decision_data)

In [14]:
# RA (Risk-Averse)
# Select the option with the highest lower limit (mean-std)

def evaluate_RA(decision_data: pd.DataFrame) -> tuple:

    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    score_history = {}  # {option: [scores]}

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        options_with_variance = []
        for option, current_score in current_options.values():
            if option in score_history and len(score_history[option]) > 1:
                s = score_history[option]
                mean = sum(s) / len(s)
                std = np.sqrt(sum((x - mean) ** 2 for x in s) / len(s))
                score = mean - std
            else:
                score = 0
            options_with_variance.append((option, score))

        min_var = max(var for _, var in options_with_variance)
        best_items = [option for option, var in options_with_variance if var == min_var]

        for option, current_score in current_options.values():
            score_history.setdefault(option, []).append(current_score)

        results = record_choice(row['Question'], best_items, current_options, results)
        
    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_RA(decision_data)

In [15]:
# RS (Risk-Seeking)
# Select the option with the highest upper limit (mean + std)

def evaluate_RS(decision_data: pd.DataFrame) -> tuple:

    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
    score_history = {}  # {option: [scores]}

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)

        options_with_variance = []
        for option, current_score in current_options.values():
            if option in score_history and len(score_history[option]) > 1:
                s = score_history[option]
                mean = sum(s) / len(s)
                std = np.sqrt(sum((x - mean) ** 2 for x in s) / len(s))
                score = mean + std
            else:
                score = 0
            options_with_variance.append((option, score))

        max_var = max(var for _, var in options_with_variance)
        best_items = [option for option, var in options_with_variance if var == max_var]

        for option, current_score in current_options.values():
            score_history.setdefault(option, []).append(current_score)

        results = record_choice(row['Question'], best_items, current_options, results)

    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_RS(decision_data)

In [16]:
# PD (Preference-Driven)
# Select the option with the highest preference value.

def evaluate_Preference(decision_data: pd.DataFrame, user_preferences: pd.DataFrame) -> tuple:

    score_history = {}
    results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])

    for index, row in questions.iterrows():
        current_options = get_current_options(row, scores, index)
        
        # Construct the preference matrix for the current problem
        preference_matrix = []
        for option, current_score in current_options.values():
            pref_score = user_preferences.get(option)
            preference_matrix.append((option, pref_score))
        
        # Select the highest preference value
        max_pref = max(pref for _, pref in preference_matrix)
        best_options = [option for option, pref in preference_matrix if pref == max_pref]
        for option, current_score in current_options.values():
            score_history.setdefault(option, []).append(current_score)
        results = record_choice(row['Question'], best_options, current_options, results)
        
    overlap_rate = calculate_overlap(decision_data.iloc[:, 2].tolist(), results['Choice'].tolist())
    first5_overlap_rate = calculate_overlap(decision_data.iloc[:5, 2].tolist(), results['Choice'][:5].tolist())
    mid15_overlap_rate = calculate_overlap(decision_data.iloc[5:20, 2].tolist(), results['Choice'][5:20].tolist())
    last20_overlap_rate = calculate_overlap(decision_data.iloc[20:, 2].tolist(), results['Choice'][20:].tolist())
    
    return (f"{overlap_rate:.2%}", f"{first5_overlap_rate:.2%}", f"{mid15_overlap_rate:.2%}", f"{last20_overlap_rate:.2%}")
evaluate_Preference(decision_data, user_preferences)

In [18]:
strategies = [
    ("Q-RL", evaluate_q_rl),
    ("R-Item", evaluate_R_Item),
    ("R-Cat", evaluate_Simple_R_Cat),
    ("LSC", evaluate_LSC),
    ("WSLS", evaluate_WSLS),
    ("EV", evaluate_EV),
    ("RA", evaluate_RA),
    ("RS", evaluate_RS),
    ("PD", evaluate_Preference)  # include user_preferences
]

# Initialization
cols = []
for name, _ in strategies:
    for suffix in ["_all", "_first5", "_mid15", "_last20"]:
        cols.append(name + suffix)

df_out = pd.DataFrame(index=range(1, 54), columns=cols)

for i in range(1, 54):
    decision_data = decision_datas[decision_datas.iloc[:, 0] == i]

    for name, func in strategies:
        if name == "PD":
            user_preferences = preferences.iloc[i-1].to_dict()
            res = func(decision_data, user_preferences)
        else:
            res = func(decision_data)

        p_all, p_first5, p_mid15, p_last20 = res
        df_out.loc[i, f"{name}_all"] = p_all
        df_out.loc[i, f"{name}_first5"] = p_first5
        df_out.loc[i, f"{name}_mid15"] = p_mid15
        df_out.loc[i, f"{name}_last20"] = p_last20

# Save as CSV
df_out.to_csv("strategy_scores.csv", index_label="participant_id")


In [24]:
df = pd.read_csv("strategy_scores.csv", index_col="participant_id")

# The strategy sequence (in line with the order of CSV columns) is saved as a CSV file
strategies_order = ["Q-RL", "R-Item", "R-Cat", "LSC", "WSLS", "EV", "RA", "RS", "PD"]
suffixes = ["all", "first5", "mid15", "last20"]

# The strategies to be deleted in the "first5" category
# Because these strategies were not sufficient to make a choice based on the information provided in the first five questions
# For example, there is a lack of standard deviation or the word only appears once
to_remove = {"EV", "RA", "RS", "Q-RL", "R-Item"}

for pid in range(1, 54):

    row = df.loc[pid]
    score_sets = {}

    # Read and convert the percentage to a floating-point number
    for suf in suffixes:
        cols = [f"{st}_{suf}" for st in strategies_order]
        raw_vals = row[cols].astype(str)
        float_vals = raw_vals.str.replace("%", "", regex=False).astype(float) / 100.0
        score_sets[suf] = float_vals.values

    # First 5 Strategy Modification (Delete 5 Strategies)
    strategies_first5 = [s for s in strategies_order if s not in to_remove]
    values_first5 = [score_sets["first5"][strategies_order.index(s)] for s in strategies_first5]

    # Angle (radar chart)
    angles_all = np.linspace(0, 2 * np.pi, len(strategies_order), endpoint=False).tolist()
    angles_first5 = np.linspace(0, 2 * np.pi, len(strategies_first5), endpoint=False).tolist()

    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(12, 10), subplot_kw=dict(polar=True))
    axes = axes.flatten()
    titles = ["All Trials", "First 5 Trials", "Middle 15 Trials", "Last 20 Trials"]

    for ax, suf, title in zip(axes, suffixes, titles):

        if suf == "first5":
            values = values_first5[:]
            strategies = strategies_first5
            ang = angles_first5[:]
        else:
            values = list(score_sets[suf])
            strategies = strategies_order
            ang = angles_all[:]

        values += values[:1]
        ang = ang + ang[:1]

        ax.plot(ang, values, linewidth=2)
        ax.fill(ang, values, alpha=0.25)

        ax.set_xticks(ang[:-1])
        ax.set_xticklabels(strategies, fontsize=9)

        ax.set_ylim(0, 1)
        ax.set_yticklabels([])
        ax.set_title(title, fontsize=13)

    fig.suptitle(f"Participant {pid} — Strategy Fit Radar Charts", fontsize=16, y=1.02)
    plt.savefig(f"strategy_fit_degree/participant_{pid}.png")
    plt.tight_layout()
    plt.show()
